In [7]:
import os
import requests
import json
import openai

openai.api_key = ""
openai.api_base = ""
openai.api_type = ''
openai.api_version = ''


In [3]:
data = pd.read_csv('data/mc.csv',index_col=0)
data.head()

,ind,id,activity_label,ctx_a,ctx_b,ctx,dataset,split_type,ending_options,label,first_part,slot_question,slot_answer
0,19073,31450,Personal Care and Style,[header] How to get perfect eyebrows [title] F...,NaN,[header] How to get perfect eyebrows [title] F...,wikihow,zeroshot,['Measure the length of your inner brow and th...,3,[header] How to get perfect eyebrows [title] F...,[header] How to get perfect eyebrows [title] F...,It should create a slight curve out the corner...
1,20017,33062,Personal Care and Style,[header] How to make french knot easy way hair...,NaN,[header] How to make french knot easy way hair...,wikihow,zeroshot,"[""For this reason, french knot style hair will...",3,[header] How to make french knot easy way hair...,[header] How to make french knot easy way hair...,"For this reason, french knot style hair will b..."
2,34560,v_pHAIrx_cRFU,Sharpening knives,The person puts a new belt on the machine. The...,the person,The person puts a new belt on the machine. The...,activitynet,zeroshot,"['puts the sword back on the machine.', 'remov...",3,The person puts a new belt on the machine,The person puts a new belt on the machine. The...,puts the sword back on the machine.
3,41427,68173,Family Life,[header] How to plan your own funeral [title] ...,NaN,[header] How to plan your own funeral [title] ...,wikihow,zeroshot,['[title] Prepare for your funeral by creating...,3,[header] How to plan your own funeral [title] ...,[header] How to plan your own funeral [title] ...,It might be something simple like the experien...
4,9755,16080,Health,[header] How to buy an exercise bike [title] E...,NaN,[header] How to buy an exercise bike [title] E...,wikihow,indomain,['You will want to check with your trainer bef...,3,[header] How to buy an exercise bike [title] E...,[header] How to buy an exercise bike [title] E...,They get you working less because you are doin...


In [39]:
ans_list = {}

In [45]:
data = data.drop(32)
data = data.reset_index(drop=True)

In [46]:
for index, row in data[32:].iterrows():
    init_prompt = "Help me complete the missing option in []. Do NOT copy other option.\n"
    prompt = init_prompt + f"{row['slot_question']}\n Answer:"
    # print(prompt)
    try:
        response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
    except openai.error.RateLimitError:
            time.sleep(10)
            response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
    # print(prompt)
    print(index,response['choices'][0]['message']['content'])
    ans_list[row['slot_question']] = response['choices'][0]['message']['content']

32 struggles a bit but eventually hoists it above his head.
33 Give your hair a weekly deep conditioning treatment. Regular deep conditioning can improve your hair's texture, making it softer and shinier.
34 option2: [additional tips] If you want an edgier look, try pairing your sheer top with a crop top or a bralette. Choose pieces with exciting elements like lace, prints, or cut-outs to make your outfit more captivating.
35 option2: Narcissists tend to be overly concerned about their appearance and aim to be seen as superior. Notice any changes like: [substeps] Becoming excessively vain or displaying arrogance and dismissive attitudes towards those they view as beneath them.
36 option0: [title] Install Memtest86+. [step] After downloading, locate and run the installer to prepare for the application.
37 option2: Consider alternatives such as LED lights. [substeps] They are energy-efficient and come in a variety of color temperatures to suit your child's needs. Regularly check the brig

KeyError: 'content'

In [38]:
data['pred_answer'] = ans_list.values()
data.to_csv("result/mc_gpt_35.csv")

In [59]:
data.to_csv("result/mc_gpt_4.csv")